<h1 style="text-align: center; color: navy; font-size: 36px;">
📢 Steganography - Hiding Messages in Images
</h1>
<h3 style="text-align: center; color:gray; font-size: 25px;">
XOR Encryption + Base64 + Bit‑Level Embedding via LSB
</h3>
<h3 style="text-align: center; color: black;">by Shaik Nihal</h3>


In [82]:
# --- Required Libraries ---
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
from PIL import Image, ImageTk  # For image handling and GUI preview
import cv2  # For image processing and pixel manipulation
import numpy as np
import base64  # For safe string encoding/decoding
import os


In [83]:
# --- Character Dictionaries for XOR Encryption ---
d = {chr(i): i for i in range(256)}  # char to ASCII
c = {i: chr(i) for i in range(256)}  # ASCII to char

In [84]:
# --- XOR Encryption & Decryption Functions ---
def xor_encrypt(text, key):
    # Encrypt each character using XOR with repeating key
    return ''.join([c[d[t] ^ d[key[i % len(key)]]] for i, t in enumerate(text)])

In [85]:

def xor_decrypt(cipher, key):
    # Decrypt XOR-encrypted text using the same key
    return ''.join([c[d[ch] ^ d[key[i % len(key)]]] for i, ch in enumerate(cipher)])

In [86]:
# --- Helper Functions ---
def str_to_bits(s):
    # Convert a list of integers (ASCII values) to a list of binary bits
    return [int(b) for ch in s for b in format(ch, '08b')]

In [87]:
def bits_to_bytes(bits):
    # Convert list of bits back into bytes (integers)
    return [int(''.join(str(bit) for bit in bits[i:i+8]), 2) for i in range(0, len(bits), 8)]

In [88]:
# --- Image Encoding Function ---
def encode_image(image_path, message, key, output_path):
    img = cv2.imread(image_path)  # Load image as array
    flat = img.flatten()  # Flatten pixel values for easy manipulation

    encrypted = xor_encrypt(message, key)  # Encrypt the message
    b64 = base64.b64encode(encrypted.encode()).decode()  # Encode to Base64
    bits = str_to_bits([ord(ch) for ch in b64])  # Convert to bits

    length = len(bits)
    len_bits = list(map(int, format(length, '032b')))  # 32-bit length header
    all_bits = len_bits + bits  # Combine length header + message bits

    # Check if the image can hold the entire message
    if len(all_bits) > len(flat):
        raise ValueError(f"Image too small to hold this message! Requires {len(all_bits)} pixels.")

    # Embed bits in the least significant bit of each pixel
    for i, bit in enumerate(all_bits):
        flat[i] = (flat[i] & 0xFE) | bit  # Replace LSB with our bit

    encoded_img = flat.reshape(img.shape)  # Reshape to original shape
    cv2.imwrite(output_path, encoded_img)  # Save encoded image
    return True

In [89]:
# --- Image Decoding Function ---
def decode_image(image_path, key):
    img = cv2.imread(image_path)
    flat = img.flatten()

    len_bits = [flat[i] & 1 for i in range(32)]  # Read the first 32 bits to get message length
    msg_len = int(''.join(str(b) for b in len_bits), 2)

    # Validate if image has enough data
    if (32 + msg_len) > len(flat):
        raise ValueError("Corrupted or invalid image. Not enough data.")

    data_bits = [flat[i] & 1 for i in range(32, 32 + msg_len)]  # Extract message bits

    if len(data_bits) % 8 != 0:
        raise ValueError("Bitstream corrupted: not divisible by 8")

    byte_vals = bits_to_bytes(data_bits)
    b64_str = ''.join(chr(b) for b in byte_vals)

    try:
        decoded_bytes = base64.b64decode(b64_str)  # Decode Base64
        decoded_encrypted = decoded_bytes.decode()  # Convert bytes to string
    except Exception as e:
        raise ValueError(f"Base64 decoding failed: {e}")

    decrypted = xor_decrypt(decoded_encrypted, key)  # Decrypt message
    return decrypted


In [90]:
# --- GUI Application ---
class StegoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🔐 Image Steganography Tool - XOR + Base64")
        self.root.geometry("800x600")

        self.img_label = None
        self.img_preview = None
        self.current_image_path = None

        # Heading and notes
        tk.Label(root, text="🛡️ Image Steganography (XOR + Base64)", font=("Arial", 16)).pack(pady=10)
        tk.Label(root, text="📢 Use clear PNG images only. Avoid JPEG to preserve accuracy.", fg="red").pack()

        # Buttons for actions
        tk.Button(root, text="🔐 Hide Message in Image", width=40, command=self.gui_encode).pack(pady=15)
        tk.Button(root, text="🔓 Reveal Message from Image", width=40, command=self.gui_decode).pack(pady=10)

        # Area for image preview
        self.img_label = tk.Label(root, bg="lightgrey")
        self.img_label.pack(pady=20)

    def show_preview(self, path):
        try:
            pil_img = Image.open(path)
            img_w, img_h = pil_img.size

            max_size = (400, 400)  # Limit preview size
            if img_w > max_size[0] or img_h > max_size[1]:
                pil_img.thumbnail(max_size, Image.LANCZOS)  # Resize while keeping aspect ratio

            self.img_preview = ImageTk.PhotoImage(pil_img)
            self.img_label.configure(image=self.img_preview, text="")
            self.img_label.image = self.img_preview  # Keep reference to avoid garbage collection
            self.img_label.config(width=pil_img.width, height=pil_img.height)
        except Exception as e:
            self.img_label.configure(text=f"Image preview failed: {e}", image="")

    def gui_encode(self):
        # Select input image
        image_path = filedialog.askopenfilename(title="Select PNG Image", filetypes=[("PNG Images", "*.png")])
        if not image_path:
            return
        self.current_image_path = image_path
        self.show_preview(image_path)

        # Ask for message and key
        message = simpledialog.askstring("Message", "Enter message to hide:", parent=self.root)
        key = simpledialog.askstring("Key", "Enter secret key:", parent=self.root)
        if not message or not key:
            messagebox.showerror("Error", "Both message and key are required.")
            return

        # Ask where to save the output
        save_path = filedialog.asksaveasfilename(defaultextension=".png", title="Save Output Image")
        try:
            encode_image(image_path, message, key, save_path)
            messagebox.showinfo("Success", f"Message successfully hidden in:\n{os.path.basename(save_path)}")
            self.show_preview(save_path)
        except Exception as e:
            messagebox.showerror("Encoding Error", str(e))

    def gui_decode(self):
        # Select encoded image
        image_path = filedialog.askopenfilename(title="Select PNG Image", filetypes=[("PNG Images", "*.png")])
        if not image_path:
            return
        self.current_image_path = image_path
        self.show_preview(image_path)

        # Ask for decryption key
        key = simpledialog.askstring("Key", "Enter decryption key:", parent=self.root)
        if not key:
            messagebox.showerror("Error", "Decryption key is required.")
            return
        try:
            message = decode_image(image_path, key)
            messagebox.showinfo("Decrypted Message", f"🔓 {message}")
        except Exception as e:
            messagebox.showerror("Decoding Error", str(e))

In [91]:
# --- Launch the App ---
def launch_gui():
    root = tk.Tk()
    app = StegoApp(root)
    root.mainloop()

In [93]:

# --- Entry Point ---
if __name__ == "__main__":
    launch_gui()
